# HHL Algorithm

The quantum algorithm for linear systems of equations, also called HHL algorithm, designed by Aram Harrow, Avinatan Hassidim, and Seth Lloyd, is a quantum algorithm formulated in 2009 for solving linear systems.
The algorithm estimates the result of a scalar measurement on the solution vector to a given linear system of equations.

Provided the linear system is sparse and has a low condition number k, and that the user is interested in the result of a scalar measurement on the solution vector, instead of the values of the solution vector itself, then the algorithm has a runtime of `O(log(N)k^2)` where N is the number of unknowns in the system.
This offers an exponential speedup over the fastest classical algorithm, which runs in `O(Nk)`.

The first demonstration of a general-purpose version of the algorithm appeared in 2018 in the work of Zhao et al.

<img src="./res/hhl.png" alt="Controlled U" width="600"/>

In this exercise we will demonstrate the algorithm for solving linear systems by Harrow, Hassidim, Lloyd (HHL).

The HHL algorithm solves a system of linear equations, specifically equations of the form `Ax = b`, where A is a Hermitian matrix, b is a known vector, and x is the unknown vector.

The algorithm uses 3 sets of qubits: a single ancilla qubit, a register (to store eigenvalues of A), and memory qubits (to store |b> and |x>).

The following are performed in order:
 1. Quantum phase estimation to extract eigenvalues of A
 2. Controlled rotations of ancilla qubit
 3. Uncomputation with inverse quantum phase estimation

The result is perfect if:
 * Each eigenvalue of the matrix is in the form `2π/t * k/N`, where `0≤k<N`, and `N=2^n`, where n is the register size. In other words, k is a value that can be represented exactly by the register.
 * `C ≤ 2π/t * 1/N`, the smallest eigenvalue that can be stored in the circuit.

The result is good if the register size is large enough such that for every pair of eigenvalues, the ratio can be approximated by a pair of possible register values.

One way to set t is `t = 2π/(sN)`

## Exercise
To simulate this circuit, we need to create several custom gates composed of multiple basic gates.
Custom gates are explained in depth in the [bonus section](/6-bonus/README.md).
Please go over it before, if you want a more indepth understanding of the implementation.

```
(0, 0): ─────────────────────────Ry(θ₄)─Ry(θ₁)─Ry(θ₂)─Ry(θ₃)──────────────M──
                     ┌──────┐    │      │      │      │ ┌───┐
(1, 0): ─H─@─────────│      │──X─@──────@────X─@──────@─│   │─────────@─H────
           │         │QFT^-1│    │      │      │      │ │QFT│         │
(2, 0): ─H─┼─────@───│      │──X─@────X─@────X─@────X─@─│   │─@───────┼─H────
           │     │   └──────┘                           └───┘ │       │
(3, 0): ───e^iAt─e^2iAt───────────────────────────────────────e^-2iAt─e^-iAt─
```

The algorithm can also be implemented without custom gates.

We will have to implement four custom gates:
 1. Phase Kickback
 2. Phase Estimation
 3. Hamiltonian Simulation
 4. Eigen Rotation

In [ ]:
import math
import numpy as np
import sympy
try:
    import cirq
except ImportError:
    %pip install --quiet cirq
    import cirq

### 1. Phase Kickback Gate

The Phase Kickback gate is actually a Controlled-U gate implemented for the purpose of this exercise.
This custom gate calls the C-U on all qubits.

<img src="./res/controlled-u.png" alt="Controlled U" width="400"/>

Follow the code hints and implement the Phase Kickback gate.

In [ ]:
class PhaseKickback(cirq.Gate):
    """A gate for the phase kickback stage of Quantum Phase Estimation.
    It consists of a series of controlled e^iAt gates with the memory qubit as the target and
    each register qubit as the control, raised to the power of 2 based on the qubit index.
    unitary is the unitary gate whose phases will be estimated.
    """

    def __init__(self, num_qubits, unitary):
        super(PhaseKickback, self)
        self._num_qubits = num_qubits
        self.U = unitary

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        memory = qubits.pop()
        for i, qubit in enumerate(qubits):
            # TODO: Apply the controlled U Gate raised to the power of 2^i
            yield TODO

    def _circuit_diagram_info_(self, args):
        return "Pk", "Pk", "Pk", "Pk", "Pk"

### 2. Phase Estimation Gate

The Phase Estimation gate consists of 3 elements:
 1. H gates applied on all qubits except the first and the last qubit
 2. A custom Phase Kickback gate which simulates the e^iAt operation using Controlled-U gates
 3. The inverse of the QFT, already implemented in Cirq

These can be visualized below:
```
(0, 0): ─────────────────────
                     ┌──────┐
(1, 0): ─H─@─────────│      │
           │         │QFT^-1│
(2, 0): ─H─┼─────@───│      │
           │     │   └──────┘
(3, 0): ───e^iAt─e^2iAt──────
```

Follow the code hints and implement the Phase Estimation gate.
The Phase Kickback gate is already called in the code.


In [ ]:
class PhaseEstimation(cirq.Gate):
    """A gate for Quantum Phase Estimation.
    The last qubit stores the eigenvector; all other qubits store the estimated phase,
    in big-endian.
    Args:
        num_qubits: The number of qubits of the unitary.
        unitary: The unitary gate whose phases will be estimated.
    """

    def __init__(self, num_qubits, unitary):
        self._num_qubits = num_qubits
        self.U = unitary

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        qubits = list(qubits)
        # TODO 1: Apply H gates on each qubit except the last one
        yield TODO
        # TODO 2: Apply the Phase Kickback
        yield TODO
        # TODO 3: Apply the reverse of QFT on each qubit except the last one (also disable reversing)
        yield TODO

    def _circuit_diagram_info_(self, args):
        return "QPE", "QPE", "QPE", "QPE", "QPE"

### 3. Hamiltonian Simulation Gate

A gate that represents e^iAt.
This `EigenGate` + `np.linalg.eigh()` implementation is used here purely for demonstrative purposes.

Observe the implementation of the Hamiltonian Simulation Gate.

In [ ]:
class HamiltonianSimulation(cirq.EigenGate):
    def __init__(self, A, t, exponent=1.0):
        cirq.EigenGate.__init__(self, exponent=exponent)
        self.A = A
        self.t = t
        ws, vs = np.linalg.eigh(A)
        self.eigen_components = []
        for w, v in zip(ws, vs.T):
            theta = w * t / math.pi
            P = np.outer(v, np.conj(v))
            self.eigen_components.append((theta, P))

    def _num_qubits_(self) -> int:
        return 1

    def _with_exponent(self, exponent):
        return HamiltonianSimulation(self.A, self.t, exponent)

    def _eigen_components(self):
        return self.eigen_components
    
    def _circuit_diagram_info_(self, args):
        return "Hs"

### 4. Eigen Rotation Gate

Perform a rotation on an ancilla equivalent to division by eigenvalues of a matrix.
`EigenRotation` performs the set of rotation on the ancilla qubit equivalent to division on the memory register by each eigenvalue of the matrix.
The last qubit is the ancilla qubit; all remaining qubits are the register, assumed to be big-endian.
It consists of a controlled ancilla qubit rotation for each possible value that can be represented by the register.
Each rotation is a `Ry` gate where the angle is calculated from the eigenvalue corresponding to the register value, up to a normalization factor C.

Pass through the hints step by step to implement the Eigen Rotation gate.

In [ ]:
class EigenRotation(cirq.Gate):
    def __init__(self, num_qubits, C, t):
        super(EigenRotation, self)
        self._num_qubits = num_qubits
        self.C = C
        self.t = t
        self.N = 2 ** (num_qubits - 1)

    def num_qubits(self):
        return self._num_qubits

    def _decompose_(self, qubits):
        for k in range(self.N):
            # TODO 1: Calculate the rotation of the qubit
            kGate = TODO

            # TODO 2: Xor the k and k - 1 qubits
            xor = TODO

            for q in qubits[-2::-1]:
                # TODO 3: Place X gates only if the current bit differs (check parity)
                if TODO:
                    yield TODO
                xor >>= 1

                # TODO 4: Add the ControlledGate with the rotation (kGate)
                kGate = TODO

            # TODO 5: Return the the gate
            yield TODO

    def _ancilla_rotation(self, k):
        if k == 0:
            k = self.N
        theta = 2 * math.asin(self.C * self.N * self.t / (2 * math.pi * k))
        return cirq.ry(theta)

    def _circuit_diagram_info_(self, args):
        return "EigInv", "EigInv", "EigInv", "EigInv", "EigInv"

### 5. Circuit Construction

<img src="./res/hhl.png" alt="Controlled U" width="600"/>

Construct the ciruit based on the image provided above.
 - A: The input Hermitian
 - C: Algorithm parameter
 - t: Algorithm parameter
 - register_size: The size of the eigenvalue register
 - *input_prep_gates: A list of gates to be applied to |0> to generate the desired input state |b>
 - Returns: The HHL circuit. The ancilla measurement has key 'a' and the memory measurement is in key 'm'.

Follow the code hints and implement the circuit.
Most of the code is already provided, concentrate on adding the gates.

In [ ]:
def hhl_circuit(A, C, t, register_size, *input_prep_gates):
    ancilla = cirq.LineQubit(0)
    # to store eigenvalues of the matrix
    register = [cirq.LineQubit(i + 1) for i in range(register_size)]
    # to store input and output vectors
    memory = cirq.LineQubit(register_size + 1)

    c = cirq.Circuit()
    hs = HamiltonianSimulation(A, t)
    pe = PhaseEstimation(register_size + 1, hs)

    # TODO 1: Append all the input preparation gates as a list
    c.append([TODO])

    # TODO 2: Append all gates between 1-5 in the diagram
    # 1. Apply the Phase Estimation
    # 2. Apply the Eigen Rotation (pay attention to the parameters)
    # 3. Apply the Inverse of Phase Estimation
    # 4. Measure the ancilla qubit (the key is 'a')
    c.append([
            TODO,
            TODO,
            TODO,
            TODO,
        ])

    # TODO 3: We now need to apply the F(x) gate. This is already done.
    # Apply a measurement on the memory qubit (the key is 'm')
    c.append([
            cirq.PhasedXPowGate(
                exponent=sympy.Symbol('exponent'), phase_exponent=sympy.Symbol('phase_exponent')
            )(memory),
            TODO,
        ])

    return c


### 6. HHL Algorithm Values

Simulates HHL with matrix input, and outputs Pauli observables of the resulting qubit state |x>.
Expected observables are calculated from the expected solution |x>.

We can now create some input values for the circuit.
Remember that we need to find a matrix A such that Ax = b, and calculate `x=[X, Y, Z]`.

We also need to define some additional parameters (`t` and `C`) used in the algorithm.

In [ ]:
# DO NOT MODIFY BELOW THIS LINE
# To test our code, we will use the following matrix:
A = np.array(
    [
        [4.30213466 - 6.01593490e-08j, 0.23531802 + 9.34386156e-01j],
        [0.23531882 - 9.34388383e-01j, 0.58386534 + 6.01593489e-08j],
    ]
)

# Used for e^iAt
t = 0.358166 * math.pi

# Number of qubits, 4 is enough:
num_qubits = 4

# We need to prepare the initial qubit state
input_prep_gates = [cirq.rx(1.276359), cirq.rz(1.276359)]

# These are the approximate results expected:
expected = (0.144130, 0.413217, -0.899154)

# Set C to be the smallest eigenvalue that can be represented by the circuit:
C = 2 * math.pi / (2**num_qubits * t)

print("Expected observable outputs:")
print("X =", expected[0])
print("Y =", expected[1])
print("Z =", expected[2])

### 7. HHL Algorithm Simulation

We can finally simulate the circuit.

Try running with a different number of repetitions.

Observe the difference between the expected and actual observables.

The difference shouldn't be bigger than ~0.1.

In [ ]:
# DO NOT MODIFY BELOW THIS LINE
circuit = hhl_circuit(A, C, t, num_qubits, *input_prep_gates)
simulator = cirq.Simulator()

# Cases for measuring X, Y, and Z (respectively) on the memory qubit.
params = [
    {'exponent': 0.5, 'phase_exponent': -0.5},
    {'exponent': 0.5, 'phase_exponent': 0},
    {'exponent': 0, 'phase_exponent': 0},
]

results = simulator.run_sweep(circuit, params, repetitions=5000)

print("Actual: ")
index = 0
for label, result in zip(('X', 'Y', 'Z'), list(results)):
    expectation = 1 - 2 * np.mean(result.measurements['m'][result.measurements['a'] == 1])
    print(f'{label} = {expectation}')
    if abs(expectation - expected[index]) > 0.1:
        print(f'Variation too big for {label}, try running again')
        exit(1)
    index += 1

print(circuit)

### 8. Bonus: Getting Close to the Expected Solution

We can, of course, get closer to the expected solution using means and medians.

Try running with a different number of repetitions and a different number of experiment repetitions.

In [ ]:
# DO NOT MODIFY BELOW THIS LINE
# We can now compute an average of the results to find a closer value to the expected result.

raw_results = [[], [], []]

# Vary this range:
for _ in range(60):
    results = simulator.run_sweep(circuit, params, repetitions=5000)
    index = 0
    for label, result in zip(('X', 'Y', 'Z'), list(results)):
        expectation = 1 - 2 * np.mean(result.measurements['m'][result.measurements['a'] == 1])
        raw_results[index].append(expectation)
        index += 1

# Sort each set of results
for i in range(3):
    raw_results[i].sort()

print("Median: ")
print(f'X = {raw_results[0][len(raw_results[0]) // 2]}')
print(f'Y = {raw_results[1][len(raw_results[1]) // 2]}')
print(f'Z = {raw_results[2][len(raw_results[2]) // 2]}')

print("Average: ")
print(f'X = {np.mean(raw_results[0])}')
print(f'Y = {np.mean(raw_results[1])}')
print(f'Z = {np.mean(raw_results[2])}')